# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json   # added for https://maps.googleapis.com/maps/api/place/nearbysearch/json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [44]:
# read in data from created by tje WeatherPy program 
cities_df = pd.read_csv("../WeatherPy/Output/city_weather.csv")
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bluff,-46.60,168.33,45.30,79,10,2.62,NZ,1603714706
1,Saint Paul Harbor,57.79,-152.41,48.20,74,90,17.22,US,1603714706
2,Jamestown,42.10,-79.24,45.00,100,90,3.36,US,1603714611
3,Cockburn Town,21.46,-71.14,83.35,80,90,19.84,TC,1603714707
4,Rikitea,-23.12,-134.97,75.07,84,100,19.62,PF,1603714707


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [43]:
# Access maps with unique API key / Configure gmaps.
gmaps.configure(api_key=g_key)
figure_layout = {'height':'700px',
                'width':'900px',
                'border':'1px solid black',
                'padding':'1px',
                 'margin':'0 auto 0 auto'
                }

# Store latitude and longitude in locations for heat map, then Fill NaN values with 0
locations = cities_df[["Lat", "Lng"]]
locations_clean = locations.fillna(0)
rating = cities_df["Humidity"]

# Plot Heatmap  - includes size to fit world
fig = gmaps.figure(layout = figure_layout, center = (0,0), zoom_level = 2)

# Create heat layer and add heat layer to fig.
heat_layer = gmaps.heatmap_layer(locations, weights=rating,
                                 dissipating=False, max_intensity=10, point_radius=1)
fig.add_layer(heat_layer)    
fig


Figure(layout=FigureLayout(border='1px solid black', height='700px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [22]:
# pick approxiametly 10 cities based on weather conditions using Loc to filter to ideal wather conditions
vacation_cities = cities_df.loc[(cities_df['Max Temp']<80) & (cities_df['Max Temp']>70)]
vacation_cities = vacation_cities.loc[(vacation_cities['Cloudiness']==0) & (vacation_cities['Wind Speed']< 10)]
vacation_cities.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
97,Yulara,-25.24,130.99,73.40,20,0,6.93,AU,1603714841
102,Vallenar,-28.57,-70.76,72.05,21,0,4.74,CL,1603714842
139,Andros Town,24.70,-77.77,77.00,94,0,4.70,BS,1603714908
157,Ormara,25.21,64.64,79.70,52,0,8.08,PK,1603714912
174,Hong Kong,22.29,114.16,77.00,79,0,1.99,HK,1603714741


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
# create new data frame for hotels, add hotel column,
hotel_df = vacation_cities.copy()
hotel_df['Hotel Name'] = ""

In [39]:
# set parameters for API and pull data by lat and long from Google Places
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
    }
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

for index, row in hotel_df.iterrows():  
    lat = row['Lat']
    lng = row['Lng']   
    params["location"] = f"{lat},{lng}"   # update address key for latitude

    # make request to Google
    hotels = requests.get(base_url, params=params).json()

     # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (KeyError, IndexError):
        print(row['City'], " - Missing field/result... skipping.")

Ormara  - Missing field/result... skipping.
Gazojak  - Missing field/result... skipping.


In [37]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
97,Yulara,-25.24,130.99,73.40,20,0,6.93,AU,1603714841,Desert Gardens Hotel - Ayers Rock Resort
102,Vallenar,-28.57,-70.76,72.05,21,0,4.74,CL,1603714842,Humacao Bed & Breakfast
139,Andros Town,24.70,-77.77,77.00,94,0,4.70,BS,1603714908,Hanks place
157,Ormara,25.21,64.64,79.70,52,0,8.08,PK,1603714912,
174,Hong Kong,22.29,114.16,77.00,79,0,1.99,HK,1603714741,Mini Hotel Central
309,Nicoya,10.15,-85.45,75.20,100,0,3.36,CR,1603715115,Guayacan Real
338,Chaozhou,23.67,116.64,75.20,69,0,6.71,CN,1603715181,Seven Corner View Homestay
407,Sīrjān,29.45,55.68,74.71,15,0,4.23,IR,1603715250,هتل عطرسیب سیرجان
422,Vityazevo,44.99,37.27,73.00,53,0,8.95,RU,1603715312,Duny Dzhemete Guest House
431,Gazojak,41.19,61.40,72.03,20,0,7.96,TM,1603715313,


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]  # frvonstruct rows
locations = hotel_df[["Lat", "Lng"]]

In [41]:
# Add marker layer ontop of heat map
# use to_records for creating a list of tuples
# https://www.kite.com/python/answers/how-to-convert-a-pandas-dataframe-into-a-list-of-tuples-in-python

coordinates_df = hotel_df[['Lat','Lng']]
records = coordinates_df.to_records(index=False)  # converts dataframe to a list of tuples
coordinates = list(records)
markers = gmaps.marker_layer(coordinates, info_box_content = hotel_info)

# Create a combined map
fig = gmaps.figure(layout = figure_layout, center = (0,0), zoom_level = 2)
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', margin='0 auto 0 auto', padding='1px', wi…

In [ ]:
# Here are the top cities to vacation in based on over 500 random cities and reduced to 
# ideal weather condions of temperature between 70 and 80, no cloudes and wind less than 10 mph
# on October 26, 2020